# Tool Usage Examples

This notebook demonstrates how to use the OpenAPI to Pydantic-AI tools converter with practical examples from both the AI Knowledge API and Ally Config API.

In [1]:
# Setup Python path to import from src directory
import sys
import os
from pathlib import Path

# Add the parent directory (project root) to Python path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

print(f"✅ Added {project_root} to Python path")
print(f"Current working directory: {Path.cwd()}")
print(f"Python path includes: {project_root}")

✅ Added /home/jschillb/meta_ally to Python path
Current working directory: /home/jschillb/meta_ally/examples
Python path includes: /home/jschillb/meta_ally


In [2]:
# Import required libraries
import asyncio
from src.lib.openapi_to_tools import OpenAPIToolsLoader
from pydantic_ai import Agent, RunContext
from typing import Any

# We'll use these for our examples
print("✅ Imports loaded successfully!")

✅ Imports loaded successfully!


In [3]:
print("🚀 Practical tool execution pattern:\n")

async def execute_tool_safely(tool, **kwargs):
    """
    Safely execute a tool with proper error handling
    """
    try:
        # Create a proper RunContext
        class MockRunContext:
            def __init__(self):
                self.deps = None
        
        ctx = MockRunContext()
        
        print(f"🔄 Attempting to call: {tool.name}")
        result = await tool.function(ctx, **kwargs)
        print(f"✅ Success! Result type: {type(result)}")
        return result
        
    except Exception as e:
        print(f"⚠️ Error calling {tool.name}: {str(e)[:100]}...")
        return None

🚀 Practical tool execution pattern:



In [4]:
# Load tools from Ally Config API
print("Loading Ally Config API tools...")

ally_config_loader = OpenAPIToolsLoader(
    openapi_url="https://ally-config-ui.dev.copilot.aws.inform-cloud.io/openapi.json",
    keycloak_url="https://keycloak.acc.iam-services.aws.inform-cloud.io/",
    realm_name="inform-ai",
    client_id="ally-portal-frontend-dev",
    models_filename="ally_config_api_models.py",
    regenerate_models=True  # Generate the models file
)
print("✅ Ally Config loader initialized.")
ally_config_tools = ally_config_loader.load_tools()

print(f"✅ Loaded {len(ally_config_tools)} Ally Config tools!")


Loading Ally Config API tools...
✅ Ally Config loader initialized.


Generating models file 'ally_config_api_models.py' from https://ally-config-ui.dev.copilot.aws.inform-cloud.io/openapi.json...
Successfully generated 'ally_config_api_models.py'
Created tool: get_aws_logs_api_getAWSLogs_post [POST /api/getAWSLogs]
Created tool: get_capabilities_api_capabilities_get [GET /api/capabilities]
Created tool: get_active_config_api_getActiveConfig_post [POST /api/getActiveConfig]
Created tool: validate_config_api_validateConfig_post [POST /api/validateConfig]
Created tool: set_config_api_setConfig_post [POST /api/setConfig]
Created tool: get_config_history_api_getConfigHistory_post [POST /api/getConfigHistory]
Created tool: create_endpoint_api_createEndpoint_post [POST /api/createEndpoint]
Created tool: get_endpoint_metadata_api_getEndpointMetadata_post [POST /api/getEndpointMetadata]
Created tool: update_endpoint_metadata_api_updateEndpointMetadata_post [POST /api/updateEndpointMetadata]
Created tool: delete_endpoint_api_deleteEndpoint_post [POST /api/deleteE

In [5]:
get_available_ai_models_tool = ally_config_loader.get_tool_by_operation_id("get_available_AI_models_api_getAvailableAIModels_post")
await execute_tool_safely(get_available_ai_models_tool)

🔄 Attempting to call: get_available_AI_models_api_getAvailableAIModels_post
Obtaining authorization token...


'\\wsl.localhost\Ubuntu-24.04\home\jschillb\meta_ally\examples'
CMD.EXE was started with the above path as the current directory.
UNC paths are not supported.  Defaulting to Windows directory.


Token obtained, expires at: 1761565694.871247     
✅ Success! Result type: <class 'dict'>


{'success': True,
 'error_msg': None,
 'data': [{'label': 'gpt-4o',
   'name': 'ChatGPT 4o',
   'description': 'ChatGPT 4 Omni - Use gpt-4o-latest instead!',
   'enabled': False},
  {'label': 'gpt-4o-latest',
   'name': 'ChatGPT 4o',
   'description': 'ChatGPT 4o (€€€€€)\n',
   'enabled': True},
  {'label': 'gpt-4o-mini',
   'name': 'ChatGPT 4o mini',
   'description': 'ChatGPT 4o mini (€) Retirement planned for 2025-09-15\n',
   'enabled': False},
  {'label': 'gpt-4.1-nano',
   'name': 'GPT 4.1 nano',
   'description': 'GPT 4.1 nano (€)\nThe gpt-4.1 series is the latest iteration of the gpt-4o model family. gpt-4.1-nano supports both text and vision provides at even lower cost and latency compared to the gpt-4o and gpt-4o-mini models.\nhttps://aka.ms/gpt-4-dot-1-blog\n',
   'enabled': True},
  {'label': 'gpt-4.1-mini',
   'name': 'GPT 4.1 mini',
   'description': 'GPT 4.1 mini (€€)\nThe gpt-4.1 series is the latest iteration of the gpt-4o model family. As with the previous gpt-4o-mini

In [6]:
# Load tools from Ally Config API
print("Loading Ally Config API tools...")

ai_knowledge_loader = OpenAPIToolsLoader(
    openapi_url="https://backend-api.dev.ai-knowledge.aws.inform-cloud.io/openapi.json",
    keycloak_url="https://keycloak.acc.iam-services.aws.inform-cloud.io/",
    realm_name="inform-ai",
    client_id="ally-portal-frontend-dev",
    models_filename="ai_knowledge_api_models.py",
    regenerate_models=True  # Generate the models file
)
print("✅ AI Knowledge loader initialized.")
ai_knowledge_tools = ai_knowledge_loader.load_tools()

print(f"✅ Loaded {len(ai_knowledge_tools)} AI Knowledge tools!")


Loading Ally Config API tools...
✅ AI Knowledge loader initialized.
Generating models file 'ai_knowledge_api_models.py' from https://backend-api.dev.ai-knowledge.aws.inform-cloud.io/openapi.json...
Successfully generated 'ai_knowledge_api_models.py'
Created tool: list_collections [GET /collections/]
Created tool: create_collection [POST /collections/]
Created tool: get_collection_configuration [GET /collections/{collection_id}]
Created tool: update_collection [PUT /collections/{collection_id}]
Created tool: delete_collection [DELETE /collections/{collection_id}]
Created tool: index_collection [POST /collections/{collection_id}/index]
Created tool: query [POST /collections/{collection_id}/query]
Created tool: get_fragment_counts [GET /collections/{collection_id}/fragments/counts]
Created tool: get_collection_usage [GET /collections/{collection_id}/usage]
Created tool: list_sources [GET /sources/]
Created tool: create_source [POST /sources/]
Created tool: get_source_configuration [GET /s

In [7]:
list_sources_tool = ai_knowledge_loader.get_tool_by_operation_id("list_sources")
await execute_tool_safely(list_sources_tool)

🔄 Attempting to call: list_sources
Obtaining authorization token...


'\\wsl.localhost\Ubuntu-24.04\home\jschillb\meta_ally\examples'
CMD.EXE was started with the above path as the current directory.
UNC paths are not supported.  Defaulting to Windows directory.


Token obtained, expires at: 1761565710.4569826    
✅ Success! Result type: <class 'dict'>


{'Count': 33,
 'ScannedCount': 59,
 'TotalCount': 33,
 'Items': [{'sourceId': 'Addone_Github',
   'sourceType': 'github',
   'description': 'For testing purpose',
   'public': False,
   'accessDenied': True},
  {'sourceId': 'angular_docs',
   'sourceType': 'website',
   'description': 'Official Angular Documentation',
   'public': False,
   'accessDenied': True},
  {'sourceId': 'Benchmark',
   'sourceType': 'sharepoint',
   'description': None,
   'public': False,
   'accessDenied': True},
  {'sourceId': 'Benchmark_no_htm',
   'sourceType': 'sharepoint',
   'description': None,
   'public': False,
   'accessDenied': True},
  {'sourceId': 'e2e_test_source_8235',
   'sourceType': 'website',
   'description': 'E2E Test Source for Playwright, created 16.9.2025, 15:42:01. Feel free to delete.',
   'public': True,
   'accessDenied': False},
  {'sourceId': 'e2e_test_source_9367',
   'sourceType': 'website',
   'description': 'E2E Test Source for Playwright, created 16.9.2025, 15:47:04. Feel f